In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.over_sampling import RandomOverSampler
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
import re
from scipy.stats import uniform


In [ ]:
# Load the dataset
df = pd.read_csv('/content/dataset1_utf8.csv')

# Randomly sample 10% of the dataset for faster processing
df_sampled = df.sample(frac=0.1, random_state=42)


In [ ]:
# Count number of columns
num_columns = df_sampled.shape[1]
print(f"Number of columns: {num_columns}")

# Function to preprocess text
def preprocess_text(text):
    # Remove special characters and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Convert to lowercase
    text = text.lower()
    return text

# Apply preprocessing to 'comment' column
df_sampled['processed_comment'] = df_sampled['comment'].apply(preprocess_text)

# Split dataset into features (X) and target (y)
X = df_sampled['processed_comment']
y = df_sampled['label']

Number of columns: 3


In [ ]:
# Vectorize the text data (convert text to numerical features)
vectorizer = TfidfVectorizer(max_features=1000)  # You can adjust max_features as needed
X_vec = vectorizer.fit_transform(X)

# Print shape of data before sampling
print(f"Shape of data before sampling: {X_vec.shape}, {y.shape}")

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_vec, y, test_size=0.2, random_state=42)

# Define the parameter distribution for Randomized Search
param_dist = {
    'alpha': uniform(0.0, 1.0)  # Hyperparameter for Laplace smoothing
}

Shape of data before sampling: (4114, 1000), (4114,)


In [ ]:
# Initialize the MultinomialNB classifier
nb = MultinomialNB()

# Initialize Randomized Search with cross-validation (reduced to 3 folds)
random_search = RandomizedSearchCV(estimator=nb, param_distributions=param_dist, n_iter=20, cv=3, verbose=2, n_jobs=-1, random_state=42)

# Fit Randomized Search on the training data
random_search.fit(X_train, y_train)

# Get the best parameters from Randomized Search
best_params = random_search.best_params_
print(f"Best parameters found: {best_params}")

# Initialize the MultinomialNB with the best parameters
model_best = MultinomialNB(**best_params)

# Fit the model on the training data
model_best.fit(X_train, y_train)

# Predict on the test set
y_pred_best = model_best.predict(X_test)

# Evaluate the model with the best parameters
print("Training with Best Parameters:")
print(classification_report(y_test, y_pred_best))

# Random Oversampling
oversampler = RandomOverSampler(random_state=42)

# Apply random oversampling to the training data
X_train_resampled, y_train_resampled = oversampler.fit_resample(X_train, y_train)

# Print shape of balanced data after oversampling
print(f"Shape of balanced data after random oversampling: {X_train_resampled.shape}, {y_train_resampled.shape}")

# Fit the best model on the resampled data
model_best.fit(X_train_resampled, y_train_resampled)

# Predict on the test set
y_pred_resampled = model_best.predict(X_test)

# Evaluate the model after random oversampling with the best parameters
print("Random Oversampling with Best Parameters:")
print(classification_report(y_test, y_pred_resampled))


Fitting 3 folds for each of 20 candidates, totalling 60 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=3.
  warnings.warn(


Best parameters found: {'alpha': 0.3042422429595377}
Training with Best Parameters:
              precision    recall  f1-score   support

           N       0.63      0.65      0.64       440
           P       0.58      0.56      0.57       383

    accuracy                           0.61       823
   macro avg       0.61      0.61      0.61       823
weighted avg       0.61      0.61      0.61       823

Shape of balanced data after random oversampling: (5151, 1000), (5151,)
Random Oversampling with Best Parameters:
              precision    recall  f1-score   support

           N       0.64      0.60      0.62       440
           O       0.00      0.00      0.00         0
           P       0.58      0.60      0.59       383

    accuracy                           0.60       823
   macro avg       0.40      0.40      0.40       823
weighted avg       0.61      0.60      0.60       823



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
